<a href="https://colab.research.google.com/github/kenny-vb244/DTSC-3020.020.2025/blob/main/Assignment_6_WebScraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [2]:
#Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


Dependencies installed.


### 2) Common Imports & Polite Headers

In [3]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


Common helpers loaded.


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [ ]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    raise NotImplementedError("TODO: implement q1_read_table")

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    raise NotImplementedError("TODO: implement q1_clean")

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    raise NotImplementedError("TODO: implement q1_sort_top")


In [6]:
# Q1 — Write your answer here

def q1_read_table(html: str) -> pd.DataFrame:
    tables = pd.read_html(html)
    # pick first table with >= 3 columns
    df = next(t for t in tables if t.shape[1] >= 3)
    df.columns = [str(c).strip() for c in df.columns]
    return df


def q1_clean(df: pd.DataFrame) -> pd.DataFrame:

    # Strip all string values
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].map(lambda x: x.strip() if isinstance(x, str) else x)

    # Rename columns
    df = df.rename(columns={
        "Country": "Country",
        "Alpha-2 code": "Alpha-2",
        "Alpha-3 code": "Alpha-3",
        "Numeric code": "Numeric"
    })

    # Uppercase
    if "Alpha-2" in df: df["Alpha-2"] = df["Alpha-2"].str.upper()
    if "Alpha-3" in df: df["Alpha-3"] = df["Alpha-3"].str.upper()

    # Convert Numeric to nullable int
    df["Numeric"] = pd.to_numeric(df["Numeric"], errors="coerce").astype("Int64")

    # Drop rows missing fields
    df = df.dropna(subset=["Country", "Alpha-2", "Alpha-3"])

    return df


def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    return df.sort_values("Numeric", ascending=False).head(top)


URL = "https://www.iban.com/country-codes"
raw_df = q1_read_table(URL)
clean_df = q1_clean(raw_df)

# Save CSV
clean_df.to_csv("data_q1.csv", index=False)

# Print Top 15 sort by Numeric
top15 = q1_sort_top(clean_df, 15)
print(top15)

                                               Country Alpha-2 Alpha-3  \
247                                             Zambia      ZM     ZMB   
246                                              Yemen      YE     YEM   
192                                              Samoa      WS     WSM   
244                                  Wallis and Futuna      WF     WLF   
240                 Venezuela (Bolivarian Republic of)      VE     VEN   
238                                         Uzbekistan      UZ     UZB   
237                                            Uruguay      UY     URY   
35                                        Burkina Faso      BF     BFA   
243                              Virgin Islands (U.S.)      VI     VIR   
236                     United States of America (the)      US     USA   
219                       Tanzania, United Republic of      TZ     TZA   
108                                        Isle of Man      IM     IMN   
113                                   

## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [ ]:
# --- Q2 Skeleton (fill the TODOs) ---
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    TODO: implement with BeautifulSoup on '.athing' and its sibling '.subtext'.
    """
    raise NotImplementedError("TODO: implement q2_parse_items")

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    TODO: cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    raise NotImplementedError("TODO: implement q2_clean")

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N. TODO: implement."""
    raise NotImplementedError("TODO: implement q2_sort_top")


In [10]:
# Q2 — Write your answer here

def q2_parse_items(html: str) -> pd.DataFrame:
    soup = BeautifulSoup(html, "html.parser")
    story_rows = soup.select("tr.athing")
    subtext_rows = soup.select("tr.athing + tr")

    data = []

    # Iterate through both lists simultaneously
    for story_row, subtext_row in zip(story_rows, subtext_rows):
        rank = story_row.select_one(".rank")
        rank = rank.text.replace(".", "").strip() if rank else None

        title_tag = story_row.select_one(".titleline a")
        title = title_tag.text.strip() if title_tag else None
        link = title_tag["href"] if title_tag and title_tag.has_attr("href") else None

        # Extract data from subtext row
        if subtext_row:
            points = subtext_row.select_one(".score")
            points = points.text.replace(" points", "").strip() if points else None

            user = subtext_row.select_one(".hnuser")
            user = user.text.strip() if user else None

            # comments are usually the last <a> in subtext
            comment_link = subtext_row.find_all("a")[-1]
            comments = comment_link.text.replace(" comments", "").replace(" comment", "").strip()
        else:
            points = user = comments = None


        data.append([rank, title, link, points, comments, user])

    return pd.DataFrame(data, columns=["rank", "title", "link", "points", "comments", "user"])


def q2_clean(df: pd.DataFrame) -> pd.DataFrame:

    # extract digits or return 0
    def to_int(x):
        m = re.search(r"\d+", str(x))
        return int(m.group()) if m else 0

    df["rank"] = df["rank"].apply(to_int)
    df["points"] = df["points"].apply(to_int)
    df["comments"] = df["comments"].apply(to_int)

    # Fill missing text fields
    df["title"] = df["title"].fillna("")
    df["link"] = df["link"].fillna("")
    df["user"] = df["user"].fillna("")

    return df


def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    return df.sort_values("points", ascending=False).head(top)


URL = "https://news.ycombinator.com/"
html = requests.get(URL).text

raw_df = q2_parse_items(html)
clean_df = q2_clean(raw_df)

# Save CSV
clean_df.to_csv("data_q2.csv", index=False)

# Print Top 15 by points (descending)
top15 = q2_sort_top(clean_df, 15)
print(top15)

    rank                                              title  \
12    13  YouTube Removes Windows 11 Bypass Tutorials, C...   
7      8                            Why I love OCaml (2023)   
27    28  VLC's Jean-Baptiste Kempf Receives the Europea...   
29    30                              James Watson has died   
0      1                                Why is Zig so cool?   
5      6  Myna: Monospace typeface designed for symbol-h...   
8      9                             Ruby Solved My Problem   
4      5                                How did I get here?   
2      3                       Becoming a Compiler Engineer   
1      2  Snapchat open-sources Valdi a cross-platform U...   
28    29  FAA to restrict commercial rocket launches to ...   
20    21                     Angel Investors, a Field Guide   
16    17                            Venn Diagram for 7 Sets   
17    18  Transducer: Composition, abstraction, performa...   
3      4       Mullvad: Shutting down our search proxy 